# Clasificación de géneros musicales utilizando Kmeans o Knn

## Parte 1 (preparación de datos)
Se enfoca en la estracción de características de audio de los archivos de audio que se van a utilizar, el objetivo es crear un archivo de datos csv para el entrenamiento de los modelos.

In [ ]:
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display as plt_dis
import numpy as np
import csv
import os
import tqdm
import math
import pandas as pd

In [ ]:
#lee un archivo de audio y returna un numpy array con el contenido
def read_audio_file(filename:str, sample_rate:None):
    data, sr = librosa.load(filename, sr=sample_rate)
    return data, sr

# permite extraer características a partir de una señal de audio y retorna
# una medida de tendencia de los datos por cada uno de los valores claves
def get_features_form_audio_data(audio_data, sample_rate, method='median'):
    rmse = librosa.feature.rms(y=audio_data)
    chroma_stft = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
    spec_cent = librosa.feature.spectral_centroid(y=audio_data, sr=sample_rate)
    spec_bw = librosa.feature.spectral_bandwidth(y=audio_data, sr=sample_rate)
    rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sample_rate)
    zcr = librosa.feature.zero_crossing_rate(audio_data) #utilizar suma en vez de media
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sample_rate)
    mfcc_array = []
    if method == 'median':
        rmse, chroma_stft, spec_cent, spec_bw, rolloff, zcr = np.median(rmse), np.median(chroma_stft), \
            np.median(spec_cent), np.median(spec_bw), np.median(rolloff), np.sum(zcr)
        for freq in mfcc:
            mfcc_array.append(np.median(freq))
    else:
        rmse, chroma_stft, spec_cent, spec_bw, rolloff, zcr = np.mean(rmse), np.mean(chroma_stft), \
            np.mean(spec_cent), np.mean(spec_bw), np.mean(rolloff), np.sum(zcr)
        for freq in mfcc:
            mfcc_array.append(np.mean(freq))
    first, *others = mfcc_array #just for testing could be updated
    return [rmse, chroma_stft, spec_cent, spec_bw, rolloff, zcr, first, others]

def generate_spectogram_image(filename:str, audio_data:np.array, genre:str, image_folder:str='./spectogram_image'):
    if not os.path.exists(image_folder):
        os.mkdir(image_folder)
    if not os.path.exists(f'{image_folder}/{genre}'):
        os.mkdir(f'{image_folder}/{genre}')
    
    fig, ax = plt.subplots(figsize=(8,8))
    spectogram_matrix = librosa.amplitude_to_db(np.abs(librosa.stft(audio_data)), ref=np.max)
    colormesh = plt_dis.specshow(spectogram_matrix ,y_axis='linear', x_axis='time', sr=sr, cmap='inferno', ax=ax)
    plt.savefig(f'{image_folder}/{genre}/{filename[:-3].replace(".", "")}.png')
    plt.clf()
    plt.close('all')

def get_section_from_audio(audio_data, sr, seconds = 30):
    dur = math.ceil(librosa.get_duration(y=audio_data, sr=sr))
    steps = (dur // seconds) - 1
    section = len(audio_data) // steps
    for i in range(steps):
        yield audio_data[section*i: section*(i+1)], f'part_{i}'
    
    

In [ ]:
AUDIO_FILE = './assets/Audio/Death on the Balcony - Tempt Of Fate.wav'
audio_data, sr = read_audio_file(AUDIO_FILE, 44100)
ipd.Audio(audio_data, rate=sr)

In [ ]:
max(audio_data), min(audio_data)

In [ ]:
mfcc = librosa.feature.mfcc(y=audio_data, sr=sr)
mfcc.shape

Debido a la duración del audio este tipo de análisis se hacen complejos por este motivo se propone descomponer el audio en secciones para ello se utiliza la función `get_section_from_audio` que obtiene partes del audio de manera secuencial para generar la data.

In [ ]:
plt_dis.waveshow(audio_data, sr=sr)

In [ ]:
spectogram_matrix = librosa.amplitude_to_db(np.abs(librosa.stft(audio_data)), ref=np.max)
plt_dis.specshow(spectogram_matrix ,y_axis='linear', x_axis='time', sr=sr)

## Extracción de variables
Se crea un proceso en el cual a partir de los audios almacenados en la carpeta `assets/Audio/` son procesados en secciones de 30 segundos, para obtener variables que nos servirán para el análisis, de clasificación, este acercamiento se basa en la idea, en que este género musical que se analiza, tiene un patrón repetitivo durante toda la canción por lo que se espera que las partes analizadas tengan un patrón similar.

In [ ]:
BASE = './assets/Audio'
header = "filename;part;chroma_stft;rmse;spectral_centroid;spectral_bandwidth;rolloff;zero_crossing_rate"
for i in range(1, 21):
    header += f';mfcc{i}'
header += ';label'
header = header.split(';')

with open('out_dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(header)

label = 0
for filename in tqdm.tqdm(os.listdir(f'{BASE}')) :
    complete_filename = f'{BASE}/{filename}'
    audio_data, sr = read_audio_file(complete_filename, 44100)
    section_generator = get_section_from_audio(audio_data,sr,15) 
    for section, part in section_generator:
        
        temp_filename = filename[:-3].replace(".", "") + '_' + part + '.wav'
        #generar imagen de espectograma (futuros análisis)
        generate_spectogram_image(temp_filename, section, 'spectograms', './assets')
        
        #extraer características del audio para crear un archivo csv
        row = [filename, part] + get_features_form_audio_data(section, sr)
        str_row = ";".join([str(x) for x in row[:-1]])+ ";"
        str_row += ";".join([str(x) for x in row[-1]])
        str_row += f";{str(label)}"
        
        with  open('out_dataset.csv', 'a', newline='') as file:
            writer = csv.writer(file, delimiter=';')
            writer.writerow(str_row.split(';'))
    label = label + 1
        

A esta data se le agrega una clasificación que se hizo a priori en el cual hay 4 posibles clasificaciones que se tomaron de manera manual en 4 grupos:

|cluster | nombre | descripcion |
|---|---|--------|
| 1  |tranqui   | diria que son ritmos más para actividades tranquilas pero que mantienen el ritmo de ambiente de dance, no son lo suficientemente "upbeat" para inspirar fiesta pero si se podría llegar a mezclar con el grupo número 2 |
| 2  |dance   | para escuchar por amantes del género en cualquier ambiente |
| 3  |muy relax   | música dance pero para actividades tranquilas como estudio |
| 4  |fiesta | música para escuchar en un ambiente bailable |

In [ ]:
df = pd.read_csv('./out_dataset.csv', delimiter=';')
df_clas = pd.read_excel('./clasificacion a priori.xlsx')

In [ ]:
data = df_clas.iloc[:, [0, 4]].merge(df, how='right', left_on='Unnamed: 0', right_on='filename')

In [ ]:
data.drop_duplicates(subset=['filename']).groupby(by='grupo', as_index=False).count().loc[:, ['grupo', 'filename']]

## Detección de variables importantes por cluster

Para cada uno de los grupos se busca determinar cuales son las variables que los representan mejor... *pendiente* para grupo 1 y 4

## Parte 2

tomar el archivo de entrada para entrenar los modelos de calsificación, examinaremos la data y realizaremos escalamiento en caso de ser necesario para algunas variables.

In [ ]:
df = pd.read_csv('./out_dataset.csv', delimiter=';')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['mfcc1'].plot(kind='box', vert=False)

In [ ]:
ax = df['mfcc1'].plot(kind='density')
ax.axvline(df['mfcc1'].mean(), color="red")
ax.axvline(df['mfcc1'].median(), color="green")

In [ ]:
df['mfcc1'].plot(kind='hist')

In [ ]:
df.corr()

## Parte 3 modelo de clasificación


In [ ]:
df_clas = df.iloc[:, 2:-1]
df_clas.head()

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [ ]:
df_clas = pd.DataFrame(normalize(df_clas), columns=df_clas.columns).drop_duplicates() #para modelos de clasificación valores duplicados son ruido

kmeans = [KMeans(n_clusters=i) for i in range(1, 20)]
score = [kmeans[i].fit(df_clas).score(df_clas) for i in range(len(kmeans))]


plt.plot(range(1, 20),score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.axvline(x = 4, color = 'r', label = 'punto de eficiencia')
plt.axvline(x = 3, color = 'r', label = 'punto de eficiencia')
plt.title('Elbow Curve')
plt.show()

Es un resultado interesante porque nos muestra que hay entre 3 o 4 grupos que se pueden extraer de la data que coinciden con la clasificación a priori que se realizó en el paso 2

In [ ]:
model = KMeans(n_clusters=4)
model.fit(df_clas)
yhat = model.predict(df_clas)

In [ ]:
centers = model.cluster_centers_
centers = pd.DataFrame(centers)
centers.columns = df_clas.columns
centers

In [ ]:
data2 = data.merge(df_clas, left_index=True, right_index=True)
data2['yhat'] = yhat
data2['grupo'] = data2['grupo']-1

In [ ]:
np.mean(data2['yhat'] == data2['grupo'])

## Se descarta KNN

Por estos datos lo mejor es continuar con un acercamiento automático

In [ ]:
X = df_clas
y = data2['yhat']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=12) #replicability
error = []
for i in range(1, 25):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error.append(np.mean(pred_i != y_test))

#graficar resultados
plt.figure(figsize=(12, 6))
plt.plot(range(1, 25), error, color='black', marker='x',markersize=10)
plt.title('Error del K medio (variables numericas)') #knn trabaja mejo
plt.xlabel('Valor de K')
plt.ylabel('Error medio')